In [116]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import scale

In [117]:
rel_count = pd.read_csv('relative_frequencies.csv')

Читаем файл нормализованных нграм

In [324]:
df = pd.read_pickle("norm-unigrams_g.pkl")
df.reset_index(inplace = True)
df_new = df.groupby(['unigram', 'year']).page_count.sum().unstack().reset_index()

In [325]:
df.unigram.nunique()

70711

In [412]:
df[df.unigram == "гуманоид"]

,unigram,year,page_count,volume_count
2937992,гуманоид,1928,3,3
2937993,гуманоид,1948,1,1
2937994,гуманоид,1955,8,6
2937995,гуманоид,1963,1,1
2937996,гуманоид,1965,1,1
2937997,гуманоид,1966,13,11
2937998,гуманоид,1967,3,3
2937999,гуманоид,1968,4,4
2938000,гуманоид,1969,5,5
2938001,гуманоид,1970,8,8


In [326]:
df_new = df_new.fillna(0)
df_new['freq_sum'] = df_new.iloc[:,1:].sum(axis=1)


rel_count = rel_count[rel_count.year > 1917]
rel_freq_list = list(rel_count.match_count)
# relative_counts = df_new.iloc[:, 1:-1] / rel_freq_list * 10000
relative_counts = df_new.iloc[:, 1:-1] / rel_freq_list
for i in relative_counts:
    df_new[i] = relative_counts[i]
    
    
result_df = df_new[df_new["freq_sum"] > 40] 
result_df = result_df.sort_values('freq_sum', ascending=False).reset_index()

In [227]:
result_df[result_df.unigram == "артель"]

year,index,unigram,1918,1919,1920,1921,1922,1923,1924,1925,...,2001,2002,2003,2004,2005,2006,2007,2008,2009,freq_sum
137,54795,артель,0.000019,0.000034,0.000027,0.000034,0.000024,0.00002,0.000035,0.000028,...,0.000009,0.000009,0.00001,0.000013,0.000012,0.000011,0.000011,0.000008,0.000009,976612.0


In [327]:
window_mean = result_df.iloc[:, 2:-1].rolling(window=5, axis=1).mean()
window_mean_df = pd.concat([result_df.iloc[:, 0:2], window_mean, result_df.iloc[:, -1]], axis=1)

rel_diff_window_mean = window_mean.diff(axis=1)
rel_diff_window_mean = rel_diff_window_mean[rel_diff_window_mean > 0]

In [328]:
z_scores_window_mean = rel_diff_window_mean.apply(lambda ngram: 
                                             scale(ngram,
                                                   axis=0,
                                                   with_mean=True, 
                                                   with_std=True,
                                                   copy=False),
                                             axis=1)

In [329]:
z_scores_window_mean_df = pd.DataFrame.from_records(z_scores_window_mean, 
                                                    columns = [str(i) for i in range(1918, 2009 + 1)])
z_scores_window_mean_df = z_scores_window_mean_df[z_scores_window_mean_df > 1.65] 
z_scores_window_mean_df = pd.concat([result_df.iloc[:, 0:2], 
                                     z_scores_window_mean_df, 
                                     result_df.iloc[:, -1]], axis=1)
z_scores_window_mean_df.fillna(0, inplace=True)

In [332]:
z_scores_window_mean_df[z_scores_window_mean_df.unigram == "гарем"]

,index,unigram,1918,1919,1920,1921,1922,1923,1924,1925,...,2001,2002,2003,2004,2005,2006,2007,2008,2009,freq_sum
960,8273,гарем,0.0,0.0,0.0,0.0,0.0,0.0,3.998461,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,79586.0


In [333]:
filtered_unigrams = {i:[] for i in list(z_scores_window_mean_df.unigram)}
for i in z_scores_window_mean_df.index:
    years_df = z_scores_window_mean_df.iloc[i, 2:-1][z_scores_window_mean_df.iloc[i, 2:-1] != 0]
    years = list(years_df.index)
    if years != []:
        filtered_unigrams[z_scores_window_mean_df.iloc[i, 1]] = years

In [334]:
window_mean = result_df.iloc[:, 2:-1].rolling(window=5, axis=1).mean()
window_mean_df = pd.concat([result_df.iloc[:, 0:2], window_mean, result_df.iloc[:, -1]], axis=1)

rel_diff_window_mean = window_mean.diff(axis=1)
rel_diff_window_mean = rel_diff_window_mean[rel_diff_window_mean > 0]

In [335]:
rel_diff_window_mean.fillna(0, inplace=True)
rel_diff_window_mean_df = pd.concat([result_df.iloc[:, 0:2], 
                                     rel_diff_window_mean, 
                                     result_df.iloc[:, -1:]], axis=1)

In [337]:
rel_diff_window_mean_df[rel_diff_window_mean_df.unigram == "гарем"]

year,index,unigram,1918,1919,1920,1921,1922,1923,1924,1925,...,2001,2002,2003,2004,2005,2006,2007,2008,2009,freq_sum
960,8273,гарем,0.0,0.0,0.0,0.0,0.0,9.859307e-08,5.113721e-07,0.0,...,0.0,5.409207e-08,0.0,1.913294e-08,0.0,1.150584e-07,0.0,1.304597e-07,0.0,79586.0


In [338]:
for i in rel_diff_window_mean_df.index:
    
    target_years = []
    years_df = rel_diff_window_mean_df.iloc[i, 2:-1][rel_diff_window_mean_df.iloc[i, 2:-1] != 0]
    years = [int(i) for i in list(years_df.index)[::-1]]

    # код ниже позволяет объединить годы, 
    # где положительная разница относительных частот, 
    # в связки, и потом в периоды (с ... по ...)
    j = 0
    current_period = []
    while j < len(years)-1:
        if years[j] - years[j+1] == 1:
            current_period.append(years[j])
        elif years[j] - years[j+1] > 1:
            current_period.append(years[j])
            if current_period != []:
                target_years.append(current_period)
            current_period = []
        j += 1
    if j == len(years)-1:
        if current_period != [] and ( current_period[-1] - years[len(years)-1] == 1):
            current_period.append(years[len(years)-1])
            target_years.append(current_period)
            current_period = []
        else:
            current_period.append(years[len(years)-1])
    if current_period != []:
        target_years.append(current_period)  
                
    result_years = []
    
    for years in target_years:
        if len(years) > 1:
            start_year, end_year = years[-1], years[0]
            result_years.append([start_year, end_year])

    filtered_unigrams[rel_diff_window_mean_df.iloc[i, 1]].append(result_years)

In [411]:
filtered_unigrams["гуманоид"]

['1980',
 '1991',
 '1992',
 '2000',
 '2008',
 [[2006, 2009], [2000, 2004], [1988, 1994], [1978, 1984], [1965, 1975]]]

In [340]:
to_del = []
for k,v in filtered_unigrams.items():
    if v == [[]]:
        to_del.append(k)

for i in to_del:
    del filtered_unigrams[i]

In [341]:
len(filtered_unigrams)

62820

Читаем файл с извлеченными датами

In [343]:
with open("dates-unigrams_g.tsv") as f:
    wikidata = pd.read_csv(f, sep="\t")
wikidata = wikidata[["Unigram", "#Q", "Subject", "#P", 
             "Predicate", "Object", "Organization", 
             "Just Date", "Start Time", "End Time", "Time Point"]]

In [344]:
# удаляем строки с ненужными предикатами
for i in wikidata.index:
    if str(wikidata["Object"][i]).startswith("http:"):
        wikidata = wikidata[wikidata.index != i]

In [345]:
wikidata = wikidata.sort_values("Unigram")

In [346]:
wikidata = wikidata[wikidata["Unigram"].isin(filtered_unigrams.keys())]

In [347]:
wikidata["Unigram"].nunique()

3298

In [348]:
wikidata.fillna(0, inplace=True)

In [349]:
wikidata_list = wikidata.values.tolist()

In [350]:
wikidata_list[1000][-4:]

['0069-01-13', 0, 0, 0]

In [351]:
result_entries = []
for unigram, dates in filtered_unigrams.items():
    for entry in wikidata_list: 
        # из каждой строки в списке берем только годы, 
        # сравниваем их с годами в словаре, если есть соответствие, 
        # добавляем всю строку Викиданных в новый список
        # погрешность - 2 года: например, рост частотности нграмы в 1965 году, в Викиданных мы смотрим 1965, 1964, 1963
        if unigram == entry[0]:
            try:
                only_wiki_years = [int(str(i)[:4]) for i in entry[-4:]]
            except ValueError:
                pass
            for date in dates:
                if isinstance(date, str):
                    date = int(date)
                    for wiki_year in only_wiki_years:
                        if date == wiki_year or date-1 == wiki_year or date-2 == wiki_year or date-100 == wiki_year or date-50 == wiki_year:
                            result_entries.append([date] + entry)
                if isinstance(date, list):
                    for date_pair in date:
                        year_1, year_2 = date_pair[0], date_pair[1]
                        for year in only_wiki_years:
                            if int(year_1) <= int(year) < int(year_2):
                                result_entries.append([date_pair] + entry)

In [352]:
result_wiki_dates = pd.DataFrame(result_entries, columns = ["Year or Period", "Unigram", "#Q", "Subject", "#P", "Predicate", "Object", "Organization", 
                                  "Just Date", "Start Time", "End Time", "Time Point"])

In [353]:
result_wiki_dates["Unigram"].nunique()

942

In [354]:
result_wiki_dates_test = result_wiki_dates.drop_duplicates(subset = ["Unigram", "#Q", "Subject", "#P", "Predicate", "Object", "Organization", 
                                  "Just Date", "Start Time", "End Time", "Time Point"], keep = "first")
result_wiki_dates_test.count()

Year or Period    2063
Unigram           2063
#Q                2063
Subject           2063
#P                2063
Predicate         2063
Object            2063
Organization      2063
Just Date         2063
Start Time        2063
End Time          2063
Time Point        2063
dtype: int64

In [355]:
result_wiki_dates_test.sort_values("Unigram", inplace=True)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [356]:
result_wiki_dates_test.to_csv("unigram-result-dates_g.tsv", sep="\t", index = False)

In [415]:
result_wiki_dates_test.iloc[2050:2100, :]

,Year or Period,Unigram,#Q,Subject,#P,Predicate,Object,Organization,Just Date,Start Time,End Time,Time Point
2389,"[1970, 1975]",гютерсло,Q6218,Гютерсло,P571,дата основания/создания/возникновения,0,0,1973-01-01,0,0,0
2395,"[1992, 2000]",гютерсло,Q3771,Гютерсло,P6,глава правительства,Karl Ernst Strothmann,0,0,1985-01-01,1994-01-01,0
2394,"[1992, 2000]",гютерсло,Q3771,Гютерсло,P463,участник организации или клуба,Climate Alliance,0,0,1995-01-01,0,0
2392,2000,гютерсло,Q6218,Гютерсло,P6,глава правительства,Sven-Georg Adenauer,0,0,1999-01-01,0,0
2390,1971,гютерсло,Q3771,Гютерсло,P17,государство,Германская империя,0,0,1871-01-18,1918-11-09,0
2391,"[1992, 2000]",гютерсло,Q3771,Гютерсло,P6,глава правительства,Maria Unger,0,0,1994-01-01,2015-01-01,0
1177,"[1925, 1934]",гянджа,Q131290,Гянджа,P131,административно-территориальная единица,Елизаветпольский уезд,0,0,1840-04-22,1930-06-30,0
1179,"[1988, 1992]",гянджа,Q131290,Гянджа,P17,государство,Азербайджанская ССР,0,0,1920-04-28,1991-08-30,0
1175,1960,гянджа,Q846357,Кяпаз,P571,дата основания/создания/возникновения,0,0,1959-01-01,0,0,0
1178,"[1988, 1992]",гянджа,Q131290,Гянджа,P17,государство,Азербайджан,0,0,1991-08-30,0,0


In [196]:
result_wiki_dates[result_wiki_dates["Unigram"] == "джанаганамана"]

,Year or Period,Unigram,#Q,Subject,#P,Predicate,Object,Organization,Just Date,Start Time,End Time,Time Point
2640,1961,джанаганамана,Q83099,Гимн Индии,P577,дата публикации,0,0,1911-12-27,0,0,0


In [162]:
result_wiki_dates[result_wiki_dates["Unigram"] == "достоевский"]

,Year or Period,Unigram,#Q,Subject,#P,Predicate,Object,Organization,Just Date,Start Time,End Time,Time Point
155,"[1980, 1982]",достоевский,Q1046852,(3453) Достоевский,P575,дата открытия,0,0,1981-09-27,0,0,0


In [ ]:
result-dates.